In [ ]:
import os
import joblib
import mlflow
import numpy as np
import pickle
import logging

import xgboost as xgb
from catboost import CatBoostClassifier

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error, f1_score, accuracy_score, recall_score

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [4]:
def load_data(filename):
    data_path = '../Data/'
    path = os.path.join(data_path, filename)
    with open(path, 'rb') as f_in:
        X, y  = pickle.load(f_in)

    if X.empty or y.empty:
        logging.error(f'{filename} data is empty')
    else:
        logging.info('Data Loaded succesfully')

    return X, y

In [5]:
train = 'train.pkl'
test = 'test.pkl'

X_train, y_train = load_data(train)
X_test, y_test = load_data(test)

2025-07-04 18:20:29,838 - INFO - Data Loaded succesfully
2025-07-04 18:20:29,841 - INFO - Data Loaded succesfully


In [6]:
y_train

0         0.0
1         0.0
2         2.0
3         0.0
4         0.0
         ... 
512719    2.0
512720    2.0
512721    2.0
512722    2.0
512723    2.0
Name: Diabetes_012, Length: 512724, dtype: float64

In [7]:
X_train

,BMI,Age,Income,PhysHlth,Education,GenHlth,MentHlth,HighBP,Fruits
0,-1.268387,1.299149,0.939638,-0.486592,6.0,2.0,-0.429630,0.0,1.0
1,0.850039,-0.010516,0.939638,-0.486592,5.0,3.0,-0.429630,0.0,0.0
2,-0.663122,1.299149,-0.026012,0.086938,5.0,2.0,-0.429630,1.0,1.0
3,-0.209174,-0.992766,0.456813,-0.486592,6.0,1.0,-0.429630,0.0,1.0
4,-0.663122,1.299149,-0.026012,-0.486592,4.0,3.0,-0.429630,0.0,1.0
...,...,...,...,...,...,...,...,...,...
512719,-0.353798,0.316900,0.939638,-0.486592,6.0,3.0,-0.429630,0.0,1.0
512720,-0.057858,0.971733,-0.026012,-0.486592,6.0,3.0,-0.429630,1.0,0.0
512721,-0.511806,0.971733,0.939638,-0.486592,6.0,3.0,-0.429630,1.0,1.0
512722,0.965673,0.933129,-2.383210,-0.115425,4.0,3.0,0.443399,1.0,0.0


In [8]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("optimization_v2")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1751641337329, experiment_id='2', last_update_time=1751641337329, lifecycle_stage='active', name='optimization_v2', tags={}>

In [ ]:
def catboos_objective(params):


In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_test, label=y_test)

def xgboost_objective(params):
    with mlflow.start_run():
        mlflow.set_tag("Model", "XGboost")
        mlflow.log_params(params)

        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred_proba = booster.predict(valid) 
        y_pred = np.argmax(y_pred_proba, axis=1)

        threshold = 0.6  # pro např. třídu 1 (diabetes)
        y_pred = []

        for probas in y_pred_proba:
            if probas[1] > threshold:
                y_pred.append(1)
            else:
                y_pred.append(np.argmax(probas))

        #Logging important metrics
        score = f1_score(y_test, y_pred, average='macro')
        loss = 1 - score
        mlflow.log_metric("f1_macro", score)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        recall = recall_score(y_test, y_pred, average='macro')
        mlflow.log_metric("recall", recall)

    return {'loss': loss, 'status': STATUS_OK}

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 3, 10, 1)),
    'learning_rate': hp.loguniform('learning_rate', -5, -1.6),
    'reg_alpha': hp.loguniform('reg_alpha', -8, 0),
    'reg_lambda': hp.loguniform('reg_lambda', -7, 0),
    'min_child_weight': hp.loguniform('min_child_weight',  1, 10),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'objective': 'multi:softprob',
    'num_class': 3,
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)